In [1]:
import http.client,datetime,os,requests, time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions

In [4]:
target = {
    "https": False,
    "host": "jupyter.cs.rcos.nii.ac.jp",
    "keyword": "JupyterHub",
    "port": 80,
    "path": "hub/home",
    "allowed_statuses": [200, 301]
}

slack_url = os.environ.get('SLACK_API_URL', None)
key_event_delay = 3
response_timeout = 20
idp_username = os.environ.get('OPENIDP_USERNAME', None)
idp_password = os.environ.get('OPENIDP_PASSWORD', None)

't7Zqrag4IpuK'

In [5]:
def check_liveness(host, path, keyword, port=80, https=False, allowed_statuses=[200]): # keyword is just a placeholder
    print(host)
    conn = None
    result = ""
    liveness = True
    try:
        if https:
            conn = http.client.HTTPSConnection(host, port=port)
        else:
            conn = http.client.HTTPConnection(host, port=port)
        conn.request( "GET", "/" + path )
        response = conn.getresponse()
    except:
        error_detected = True
        return (f"{host}:{port}/{path} : failed to connect", False)
    if response.status not in allowed_statuses:
        liveness = False
    result = f"{host}/{path} {response.status} : {response.reason}"
    if conn != None:
        conn.close()
    return (result, liveness)

In [6]:
def wait_load(driver, element_id, timeout):
    try:
        return WebDriverWait(driver, timeout).until(expected_conditions.presence_of_element_located((By.CSS_SELECTOR, element_id)))
    except TimeoutException:
        raise Exception(f"Timeout occured after waiting {element_id} for {timeout} seconds.")

In [7]:
def check_by_selenium(host, path, keyword, port=80, https=False, allowed_statuses=[]): # allowed_statuses is just a placeholder
    alive = False
    try:
        driver = webdriver.Firefox()
        location = f"{host}:{port}/{path}"
        result = ""
        if https:
            driver.get("https://" + location)
        else:
            driver.get("http://" + location)
        # login 
        wait_load(driver, "#keytext", response_timeout).send_keys("OpenIdP")
        time.sleep(key_event_delay)
        driver.find_element_by_css_selector("input[type='submit']").click()
        wait_load(driver, "input[name='j_username']", response_timeout).send_keys(idp_username)
        time.sleep(key_event_delay)
        driver.find_element_by_css_selector("input[name='j_password']").send_keys(idp_password)
        time.sleep(key_event_delay)
        driver.find_element_by_css_selector("button[type='submit']").click()
        
        # This part is necessary on only the first login 
        try:
            wait_load(driver, "input[type='submit'][name='_eventId_proceed']", response_timeout).click()
        except:
            pass
        
        time.sleep(response_timeout) # long delay to get page title
        if not keyword in driver.title:
            result = f"{host}:{port}/{path} : {keyword} is not included in page title! ({driver.title})"
        alive = True
    except Exception as e:
        result = f"An exception occured on {host}:{port}/{path} : {e}"
    finally:
        driver.close()
    return result, alive

In [8]:
(message, alive) = check_liveness(**target)
if alive:
    (message, alive) = check_by_selenium(**target)

if not alive:
    now = datetime.datetime.now()
    message = str(now) + "\n" + message
    data = {'text': message}
    x = requests.post(slack_url, json=data)

jupyter.cs.rcos.nii.ac.jp
